In [3]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

#### Temasek Foundation Dataset

In [3]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_groq_response, test_cases=test_cases[:3],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [4]:
node.evolve("i1", batch_size=3) # run evolution :: Need to parallize test input inference



Code 1 outputs:
Test 1: {'label': 'Yes', 'comment': '"Marine Battery As A Service" project demonstrates a clear understanding of the problem, a comprehensive solution, and strong stakeholder engagement. However, the application misses critical information such as project objectives, grant amount sought, and proof of concept. I recommend providing more detailed project plans and financial information to support the grant application.'}
Test 2: {'label': 'Maybe', 'comment': 'Project has potential, but lacks clear objectives and proposed scope of work'}
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.44
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.22
  🔄 Compiled solutions:        3
  ⏱️ Time breakdown:
     :: Query time: 23.22s
     :: Evolution time: 4.33s
     :: Evaluation time: 15.62s
     :: Total time: 43.16s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM respon